In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
from sklearn.preprocessing import LabelEncoder
from scipy.stats import mode
import pandas_profiling
from sklearn.decomposition import PCA

from sklearn.model_selection import GridSearchCV,RandomizedSearchCV,KFold
from sklearn.metrics import classification_report,roc_auc_score,roc_curve
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier,BaggingClassifier
from sklearn.metrics import auc

import os
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_colwidth',500)
pd.set_option('display.max_columns',5000)

encoder = LabelEncoder()

In [38]:
df = pd.read_csv('final_train.csv')
df.head()

,Unnamed: 0,coupon_id,customer_id,redemption_status,brand,brand_type,category,cd_sum,coupon_discount_x,coupon_used_x,item_counts,no_of_customers,od_sum,other_discount_x,price_sum,qu_sum,quantity_x,selling_price_x,t_counts,total_discount_mean,total_discount_sum,campaign_type,campaign_duration,age_range,marital_status,rented,family_size,no_of_children,income_bracket,coupon_discount_y,coupon_used_y,day,dow,no_of_items,month,other_discount_y,quantity_y,selling_price_y,cdd_sum,customer_id_count,odd_sum,qa_sum,pprice_sum
0,0,27,1053,0,1105,0,6,-5349.93,-1.241694,191.0,125,19.224000,-177447.090167,-44.367382,436274.446738,4661.0,1.169768,117.276864,3713.0,-45.609077,-182797.020167,0,47,3.0,1.0,0.0,1.0,0.0,5.0,-0.287258,1,11,2,208,5,-25.583099,340.487097,163.966826,-89.05,310,-7930.760842,105551,50829.715972
1,1,116,48,0,56,1,6,0.00,0.000000,0.0,3,20.333333,-586.140000,-10.207037,5323.510000,86.0,1.050926,70.885046,75.0,-10.207037,-586.140000,0,47,2.0,0.0,0.0,2.0,0.0,3.0,-3.215039,12,4,5,244,6,-20.013685,31.540260,188.638474,-1237.79,385,-7705.268702,12143,72625.812434
2,2,635,205,0,560,0,11,-605.54,-2.987321,15.0,67,1.761194,-12475.083333,-76.767366,48980.998333,202.0,1.353802,342.826195,142.0,-79.754687,-13080.623333,1,32,3.0,0.0,0.0,2.0,0.0,7.0,-2.212082,85,27,2,533,3,-12.873370,1.392784,112.067666,-2145.72,970,-12487.169143,1351,108705.636349
3,3,644,1050,0,611,0,6,-17.81,-4.452500,1.0,4,3.500000,-260.750000,-4.451964,3697.783333,47.0,1.110390,88.694903,39.0,-8.904464,-278.560000,0,47,3.0,0.0,0.0,2.0,0.0,5.0,-0.751477,2,9,4,216,8,-12.880868,1.291139,100.896997,-178.10,237,-3052.765833,306,23912.588333
4,4,1017,1489,0,1558,0,6,-569.92,-0.198055,23.0,32,27.312500,-46796.046667,-30.665116,289181.848333,1610.0,1.071044,200.991719,1420.0,-30.863170,-47365.966667,0,48,3.0,0.0,0.0,2.0,0.0,3.0,-0.471548,10,2,4,327,5,-12.264174,247.443060,85.016352,-265.01,562,-6892.466021,139063,47779.189609


In [39]:
df.drop(columns=['Unnamed: 0'],inplace=True)
df.head()

,coupon_id,customer_id,redemption_status,brand,brand_type,category,cd_sum,coupon_discount_x,coupon_used_x,item_counts,no_of_customers,od_sum,other_discount_x,price_sum,qu_sum,quantity_x,selling_price_x,t_counts,total_discount_mean,total_discount_sum,campaign_type,campaign_duration,age_range,marital_status,rented,family_size,no_of_children,income_bracket,coupon_discount_y,coupon_used_y,day,dow,no_of_items,month,other_discount_y,quantity_y,selling_price_y,cdd_sum,customer_id_count,odd_sum,qa_sum,pprice_sum
0,27,1053,0,1105,0,6,-5349.93,-1.241694,191.0,125,19.224000,-177447.090167,-44.367382,436274.446738,4661.0,1.169768,117.276864,3713.0,-45.609077,-182797.020167,0,47,3.0,1.0,0.0,1.0,0.0,5.0,-0.287258,1,11,2,208,5,-25.583099,340.487097,163.966826,-89.05,310,-7930.760842,105551,50829.715972
1,116,48,0,56,1,6,0.00,0.000000,0.0,3,20.333333,-586.140000,-10.207037,5323.510000,86.0,1.050926,70.885046,75.0,-10.207037,-586.140000,0,47,2.0,0.0,0.0,2.0,0.0,3.0,-3.215039,12,4,5,244,6,-20.013685,31.540260,188.638474,-1237.79,385,-7705.268702,12143,72625.812434
2,635,205,0,560,0,11,-605.54,-2.987321,15.0,67,1.761194,-12475.083333,-76.767366,48980.998333,202.0,1.353802,342.826195,142.0,-79.754687,-13080.623333,1,32,3.0,0.0,0.0,2.0,0.0,7.0,-2.212082,85,27,2,533,3,-12.873370,1.392784,112.067666,-2145.72,970,-12487.169143,1351,108705.636349
3,644,1050,0,611,0,6,-17.81,-4.452500,1.0,4,3.500000,-260.750000,-4.451964,3697.783333,47.0,1.110390,88.694903,39.0,-8.904464,-278.560000,0,47,3.0,0.0,0.0,2.0,0.0,5.0,-0.751477,2,9,4,216,8,-12.880868,1.291139,100.896997,-178.10,237,-3052.765833,306,23912.588333
4,1017,1489,0,1558,0,6,-569.92,-0.198055,23.0,32,27.312500,-46796.046667,-30.665116,289181.848333,1610.0,1.071044,200.991719,1420.0,-30.863170,-47365.966667,0,48,3.0,0.0,0.0,2.0,0.0,3.0,-0.471548,10,2,4,327,5,-12.264174,247.443060,85.016352,-265.01,562,-6892.466021,139063,47779.189609


In [40]:
df.shape

(78369, 42)

In [41]:
from imblearn.over_sampling import SMOTE

# Separate input features and target
y = df.redemption_status
x = df.drop('redemption_status', axis=1)

# Standardizig the Data
col_names = ['cd_sum','coupon_discount_x', 'coupon_used_x', 'item_counts', 'no_of_customers',
       'od_sum', 'other_discount_x', 'price_sum', 'qu_sum', 'quantity_x',
       'selling_price_x', 't_counts', 'total_discount_mean',
       'total_discount_sum', 'campaign_type', 'campaign_duration',
        'family_size', 'no_of_children',
       'income_bracket', 'coupon_discount_y', 'coupon_used_y',
       'no_of_items', 'other_discount_y', 'quantity_y',
       'selling_price_y', 'cdd_sum', 'customer_id_count', 'odd_sum', 'qa_sum',
       'pprice_sum']
features = x[col_names]
scaler = StandardScaler().fit(features.values)
features = scaler.transform(features.values)
x[col_names] = features


# setting up testing and training sets
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=1990)

sm = SMOTE(random_state=1990, ratio=1.0)
x_train, y_train = sm.fit_sample(x_train, y_train)

In [42]:
x.head()

,coupon_id,customer_id,brand,brand_type,category,cd_sum,coupon_discount_x,coupon_used_x,item_counts,no_of_customers,od_sum,other_discount_x,price_sum,qu_sum,quantity_x,selling_price_x,t_counts,total_discount_mean,total_discount_sum,campaign_type,campaign_duration,age_range,marital_status,rented,family_size,no_of_children,income_bracket,coupon_discount_y,coupon_used_y,day,dow,no_of_items,month,other_discount_y,quantity_y,selling_price_y,cdd_sum,customer_id_count,odd_sum,qa_sum,pprice_sum
0,27,1053,1105,0,6,-0.550822,0.176138,0.711921,0.071188,0.124860,-0.648761,-1.135068,0.173037,0.128690,-0.012769,-0.309623,0.140786,-1.012073,-0.649702,-0.598825,0.362643,3.0,1.0,0.0,-1.241845,-0.436522,0.062799,0.305052,-0.449826,11,2,-1.188299,5,-3.061232,0.982589,3.466661,0.455543,-1.097975,0.638587,-0.147977,-0.790891
1,116,48,56,1,6,0.197203,0.389675,-0.203806,-0.130455,0.207049,0.157914,0.504670,-0.166087,-0.160847,-0.461816,-0.639102,-0.163867,0.571729,0.160150,-0.598825,0.362643,2.0,0.0,0.0,-0.142992,-0.436522,-1.078547,-2.345483,-0.165313,4,5,-1.077153,6,-1.623272,-0.446425,4.854179,-0.444733,-0.984004,0.662184,-0.534847,-0.471809
2,635,205,560,0,11,0.112536,-0.124061,-0.131890,-0.024675,-1.168935,0.103688,-2.690307,-0.131732,-0.153506,0.682610,1.292250,-0.158256,-2.539664,0.104617,1.669936,-2.078113,3.0,0.0,0.0,-0.142992,-0.436522,1.204145,-1.437502,1.722817,27,2,-0.184896,3,0.220277,-0.585870,0.547886,-1.156285,-0.095029,0.161766,-0.579545,0.056379
3,644,1050,611,0,6,0.194713,-0.376031,-0.199011,-0.128802,-1.040109,0.159398,0.780921,-0.167366,-0.163316,-0.237131,-0.512615,-0.166882,0.630003,0.161517,-0.598825,0.362643,3.0,0.0,0.0,-0.142992,-0.436522,0.062799,-0.115208,-0.423961,9,4,-1.163600,8,0.218341,-0.586340,-0.080346,0.385754,-1.208907,1.149062,-0.583873,-1.184941
4,1017,1489,1558,0,6,0.117517,0.355615,-0.093535,-0.082523,0.724126,-0.052853,-0.477343,0.057287,-0.064398,-0.385799,0.284928,-0.051234,-0.352376,-0.047767,-0.598825,0.525360,3.0,0.0,0.0,-0.142992,-0.436522,-1.078547,0.138213,-0.217043,2,4,-0.820899,5,0.377565,0.552219,-0.973463,0.317642,-0.715032,0.747243,-0.009180,-0.835549


In [43]:
y.value_counts()

0    77640
1      729
Name: redemption_status, dtype: int64

In [44]:
x.columns

Index(['coupon_id', 'customer_id', 'brand', 'brand_type', 'category', 'cd_sum',
       'coupon_discount_x', 'coupon_used_x', 'item_counts', 'no_of_customers',
       'od_sum', 'other_discount_x', 'price_sum', 'qu_sum', 'quantity_x',
       'selling_price_x', 't_counts', 'total_discount_mean',
       'total_discount_sum', 'campaign_type', 'campaign_duration', 'age_range',
       'marital_status', 'rented', 'family_size', 'no_of_children',
       'income_bracket', 'coupon_discount_y', 'coupon_used_y', 'day', 'dow',
       'no_of_items', 'month', 'other_discount_y', 'quantity_y',
       'selling_price_y', 'cdd_sum', 'customer_id_count', 'odd_sum', 'qa_sum',
       'pprice_sum'],
      dtype='object')

In [45]:
x.shape

(78369, 41)

In [46]:
x, y = sm.fit_sample(x, y)

In [47]:
x.shape,y.shape

((155280, 41), (155280,))

In [48]:
y

array([0, 0, 0, ..., 1, 1, 1], dtype=int64)

In [49]:
y1 = pd.DataFrame(y)

In [50]:
y1.head()

,0
0,0
1,0
2,0
3,0
4,0


In [51]:
y1.tail()

,0
155275,1
155276,1
155277,1
155278,1
155279,1


In [52]:
y1[0].value_counts(normalize=True)*100

1    50.0
0    50.0
Name: 0, dtype: float64

In [18]:
x1 = pd.DataFrame(x)

In [19]:
smote = pd.concat([x1,y1], axis=1)

smote.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,0
0,27.0,1053.0,1105.0,0.0,6.0,-0.550822,0.176138,0.711921,0.071188,0.124860,-0.648761,-1.135068,0.173037,0.128690,-0.012769,-0.309623,0.140786,-1.012073,-0.649702,-0.598825,0.362643,3.0,1.0,0.0,-1.241845,-0.436522,0.062799,0.305052,-0.449826,11.0,2.0,-1.188299,5.0,-3.061232,0.982589,3.466661,0.455543,-1.097975,0.638587,-0.147977,-0.790891,0
1,116.0,48.0,56.0,1.0,6.0,0.197203,0.389675,-0.203806,-0.130455,0.207049,0.157914,0.504670,-0.166087,-0.160847,-0.461816,-0.639102,-0.163867,0.571729,0.160150,-0.598825,0.362643,2.0,0.0,0.0,-0.142992,-0.436522,-1.078547,-2.345483,-0.165313,4.0,5.0,-1.077153,6.0,-1.623272,-0.446425,4.854179,-0.444733,-0.984004,0.662184,-0.534847,-0.471809,0
2,635.0,205.0,560.0,0.0,11.0,0.112536,-0.124061,-0.131890,-0.024675,-1.168935,0.103688,-2.690307,-0.131732,-0.153506,0.682610,1.292250,-0.158256,-2.539664,0.104617,1.669936,-2.078113,3.0,0.0,0.0,-0.142992,-0.436522,1.204145,-1.437502,1.722817,27.0,2.0,-0.184896,3.0,0.220277,-0.585870,0.547886,-1.156285,-0.095029,0.161766,-0.579545,0.056379,0
3,644.0,1050.0,611.0,0.0,6.0,0.194713,-0.376031,-0.199011,-0.128802,-1.040109,0.159398,0.780921,-0.167366,-0.163316,-0.237131,-0.512615,-0.166882,0.630003,0.161517,-0.598825,0.362643,3.0,0.0,0.0,-0.142992,-0.436522,0.062799,-0.115208,-0.423961,9.0,4.0,-1.163600,8.0,0.218341,-0.586340,-0.080346,0.385754,-1.208907,1.149062,-0.583873,-1.184941,0
4,1017.0,1489.0,1558.0,0.0,6.0,0.117517,0.355615,-0.093535,-0.082523,0.724126,-0.052853,-0.477343,0.057287,-0.064398,-0.385799,0.284928,-0.051234,-0.352376,-0.047767,-0.598825,0.525360,3.0,0.0,0.0,-0.142992,-0.436522,-1.078547,0.138213,-0.217043,2.0,4.0,-0.820899,5.0,0.377565,0.552219,-0.973463,0.317642,-0.715032,0.747243,-0.009180,-0.835549,0


In [20]:
smote.columns=['coupon_id', 'customer_id', 'brand', 'brand_type', 'category', 'cd_sum',
       'coupon_discount_x', 'coupon_used_x', 'item_counts', 'no_of_customers',
       'od_sum', 'other_discount_x', 'price_sum', 'qu_sum', 'quantity_x',
       'selling_price_x', 't_counts', 'total_discount_mean',
       'total_discount_sum', 'campaign_type', 'campaign_duration', 'age_range',
       'marital_status', 'rented', 'family_size', 'no_of_children',
       'income_bracket', 'coupon_discount_y', 'coupon_used_y', 'day', 'dow',
       'no_of_items', 'month', 'other_discount_y', 'quantity_y',
       'selling_price_y', 'cdd_sum', 'customer_id_count', 'odd_sum', 'qa_sum',
       'pprice_sum','redemption_status']

In [21]:
smote.head()

,coupon_id,customer_id,brand,brand_type,category,cd_sum,coupon_discount_x,coupon_used_x,item_counts,no_of_customers,od_sum,other_discount_x,price_sum,qu_sum,quantity_x,selling_price_x,t_counts,total_discount_mean,total_discount_sum,campaign_type,campaign_duration,age_range,marital_status,rented,family_size,no_of_children,income_bracket,coupon_discount_y,coupon_used_y,day,dow,no_of_items,month,other_discount_y,quantity_y,selling_price_y,cdd_sum,customer_id_count,odd_sum,qa_sum,pprice_sum,redemption_status
0,27.0,1053.0,1105.0,0.0,6.0,-0.550822,0.176138,0.711921,0.071188,0.124860,-0.648761,-1.135068,0.173037,0.128690,-0.012769,-0.309623,0.140786,-1.012073,-0.649702,-0.598825,0.362643,3.0,1.0,0.0,-1.241845,-0.436522,0.062799,0.305052,-0.449826,11.0,2.0,-1.188299,5.0,-3.061232,0.982589,3.466661,0.455543,-1.097975,0.638587,-0.147977,-0.790891,0
1,116.0,48.0,56.0,1.0,6.0,0.197203,0.389675,-0.203806,-0.130455,0.207049,0.157914,0.504670,-0.166087,-0.160847,-0.461816,-0.639102,-0.163867,0.571729,0.160150,-0.598825,0.362643,2.0,0.0,0.0,-0.142992,-0.436522,-1.078547,-2.345483,-0.165313,4.0,5.0,-1.077153,6.0,-1.623272,-0.446425,4.854179,-0.444733,-0.984004,0.662184,-0.534847,-0.471809,0
2,635.0,205.0,560.0,0.0,11.0,0.112536,-0.124061,-0.131890,-0.024675,-1.168935,0.103688,-2.690307,-0.131732,-0.153506,0.682610,1.292250,-0.158256,-2.539664,0.104617,1.669936,-2.078113,3.0,0.0,0.0,-0.142992,-0.436522,1.204145,-1.437502,1.722817,27.0,2.0,-0.184896,3.0,0.220277,-0.585870,0.547886,-1.156285,-0.095029,0.161766,-0.579545,0.056379,0
3,644.0,1050.0,611.0,0.0,6.0,0.194713,-0.376031,-0.199011,-0.128802,-1.040109,0.159398,0.780921,-0.167366,-0.163316,-0.237131,-0.512615,-0.166882,0.630003,0.161517,-0.598825,0.362643,3.0,0.0,0.0,-0.142992,-0.436522,0.062799,-0.115208,-0.423961,9.0,4.0,-1.163600,8.0,0.218341,-0.586340,-0.080346,0.385754,-1.208907,1.149062,-0.583873,-1.184941,0
4,1017.0,1489.0,1558.0,0.0,6.0,0.117517,0.355615,-0.093535,-0.082523,0.724126,-0.052853,-0.477343,0.057287,-0.064398,-0.385799,0.284928,-0.051234,-0.352376,-0.047767,-0.598825,0.525360,3.0,0.0,0.0,-0.142992,-0.436522,-1.078547,0.138213,-0.217043,2.0,4.0,-0.820899,5.0,0.377565,0.552219,-0.973463,0.317642,-0.715032,0.747243,-0.009180,-0.835549,0


In [23]:
smote.to_csv(r'C:\Users\Trinadh\Desktop\DSE\Problem\final\smote.csv', index=False)

In [ ]:
x_train.shape,y_train.shape,x_test.shape,y_test.shape

# Logistic Regression

In [ ]:
LR = LogisticRegression()
LR.fit(x_train,y_train)
y_pred_LR = LR.predict(x_test)
print(classification_report(y_test,y_pred_LR))

In [ ]:
print(roc_auc_score(y_test,y_pred_LR))
Model = ['Logistic Regression']
ROC_AUC_Accuracy = [roc_auc_score(y_test,y_pred_LR)]

# kNN

## Caution --- Algorithm - 'brute' was crashing the system, hence removed it.

In [ ]:
params = {
    
    'n_neighbors': range(1,5),
    'weights': ['uniform','distance'],
    'algorithm': ['ball_tree','kd_tree'],
    'p': [1,2,3]
}

knn = KNeighborsClassifier()

rs = RandomizedSearchCV(estimator=knn,n_jobs=-1,cv=3,param_distributions=params,scoring='recall')
rs.fit(x,y)

In [ ]:
knn = KNeighborsClassifier(**rs.best_params_)
knn.fit(x_train,y_train)
y_pred_knn = knn.predict(x_test)
print(roc_auc_score(y_test,y_pred_knn))
Model.append('k-Nearest-Neighbours')
ROC_AUC_Accuracy.append(roc_auc_score(y_test,y_pred_knn))

# Naive Bayes

In [ ]:
nb = GaussianNB()
nb.fit(x_train,y_train)
y_pred_nb = nb.predict(x_test)
print(roc_auc_score(y_test,y_pred_nb))
Model.append('Naive Bayes')
ROC_AUC_Accuracy.append(roc_auc_score(y_test,y_pred_nb))

In [ ]:
print(classification_report(y_test,y_pred_nb))

# Decision Tree

In [ ]:
params = {
    
    'criterion':['gini','entropy'],
    'splitter':['best','random'],
    'max_depth':range(1,10),
    'max_leaf_nodes':range(2,10,1),
    'max_features':['auto','log2']
    
}

dt = DecisionTreeClassifier()

rs = RandomizedSearchCV(estimator=dt,n_jobs=-1,cv=3,param_distributions=params,scoring='recall')
rs.fit(x,y)

In [ ]:
dt = DecisionTreeClassifier(**rs.best_params_)
dt.fit(x_train,y_train)
y_pred_dt = dt.predict(x_test)
print(roc_auc_score(y_test,y_pred_dt))
Model.append('Decision Tree')
ROC_AUC_Accuracy.append(roc_auc_score(y_test,y_pred_dt))

In [ ]:
print(classification_report(y_test,y_pred_dt))

In [ ]:
Model,ROC_AUC_Accuracy

# Random Forest

In [ ]:
params = {
    
    'n_estimators':range(10,100,10),
    'criterion':['gini','entropy'],
    'max_depth':range(2,10,1),
    'max_leaf_nodes':range(2,10,1),
    'max_features':['auto','log2']
    
}

rf = RandomForestClassifier()

rs = RandomizedSearchCV(estimator=rf,param_distributions=params,cv=3,scoring='recall',n_jobs=-1)
rs.fit(x,y)

In [ ]:
rf = RandomForestClassifier(**rs.best_params_)
rf.fit(x_train,y_train)
y_pred_rf = rf.predict(x_test)
print(roc_auc_score(y_test,y_pred_rf))
Model.append('Random Forest')
ROC_AUC_Accuracy.append(roc_auc_score(y_test,y_pred_rf))

In [ ]:
print(classification_report(y_test,y_pred_rf))

# Now using the Bagging Classifiers

In [ ]:
LR_Bag = BaggingClassifier(base_estimator=LR,n_estimators=100,n_jobs=-1,random_state=1)
knn_Bag = BaggingClassifier(base_estimator=knn,n_estimators=100,n_jobs=-1,random_state=1)
nb_Bag = BaggingClassifier(base_estimator=nb,n_estimators=100,n_jobs=-1,random_state=1)
dt_Bag = BaggingClassifier(base_estimator=dt,n_estimators=100,n_jobs=-1,random_state=1)

In [ ]:
x = np.array(x)

In [ ]:
kf = KFold(n_splits=3,shuffle=True,random_state=1)
model = LR_Bag
name = 'Bagged-LR'
#for model,name in zip([LR_Bag,knn_Bag,nb_Bag,dt_Bag],['Bagged-LR','Bagged-kNN','Bagged-NB','Bagged-DT']):
roc_acc = []
for train,test in kf.split(x,y):
    x_train = x[train,:]
    x_test = x[test,:]
    y_train = y[train]
    y_test = y[test]
    sm = SMOTE(random_state=1990, ratio=1.0)
    x_train, y_train = sm.fit_sample(x_train, y_train)
    model.fit(x_train,y_train)
    y_pred = model.predict(x_test)
    score = roc_auc_score(y_test,y_pred)
    fpr,tpr,_ = roc_curve(y_test,y_pred)
    roc_acc.append(auc(fpr,tpr))
Model.append(name)
ROC_AUC_Accuracy.append(np.mean(roc_acc))
print('The AUC Score for')
print('%s is %0.02f with variacne of (+/-) %0.5f'%(name,np.mean(roc_acc),np.var(roc_acc,ddof=1)))

In [ ]:
kf = KFold(n_splits=3,shuffle=True,random_state=1)
model = nb_Bag
name = 'Bagged-NB'
#for model,name in zip([LR_Bag,knn_Bag,nb_Bag,dt_Bag],['Bagged-LR','Bagged-kNN','Bagged-NB','Bagged-DT']):
roc_acc = []
for train,test in kf.split(x,y):
    x_train = x[train,:]
    x_test = x[test,:]
    y_train = y[train]
    y_test = y[test]
    sm = SMOTE(random_state=1990, ratio=1.0)
    x_train, y_train = sm.fit_sample(x_train, y_train)
    model.fit(x_train,y_train)
    y_pred = model.predict(x_test)
    score = roc_auc_score(y_test,y_pred)
    fpr,tpr,_ = roc_curve(y_test,y_pred)
    roc_acc.append(auc(fpr,tpr))
Model.append(name)
ROC_AUC_Accuracy.append(np.mean(roc_acc))
print('The AUC Score for')
print('%s is %0.02f with variacne of (+/-) %0.5f'%(name,np.mean(roc_acc),np.var(roc_acc,ddof=1)))

In [ ]:
kf = KFold(n_splits=3,shuffle=True,random_state=1)
model = dt_Bag
name = 'Bagged-DT'
#for model,name in zip([LR_Bag,knn_Bag,nb_Bag,dt_Bag],['Bagged-LR','Bagged-kNN','Bagged-NB','Bagged-DT']):
roc_acc = []
for train,test in kf.split(x,y):
    x_train = x[train,:]
    x_test = x[test,:]
    y_train = y[train]
    y_test = y[test]
    sm = SMOTE(random_state=1990, ratio=1.0)
    x_train, y_train = sm.fit_sample(x_train, y_train)
    model.fit(x_train,y_train)
    y_pred = model.predict(x_test)
    score = roc_auc_score(y_test,y_pred)
    fpr,tpr,_ = roc_curve(y_test,y_pred)
    roc_acc.append(auc(fpr,tpr))
Model.append(name)
ROC_AUC_Accuracy.append(np.mean(roc_acc))
print('The AUC Score for')
print('%s is %0.02f with variacne of (+/-) %0.5f'%(name,np.mean(roc_acc),np.var(roc_acc,ddof=1)))

## Now Moving Towards Boosting?

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from lightgbm import LGBMModel,LGBMClassifier

In [ ]:
LR_Boost = AdaBoostClassifier(base_estimator=LR,n_estimators=100,learning_rate=0.01,random_state=1)
knn_Boost = AdaBoostClassifier(base_estimator=knn,n_estimators=100,learning_rate=0.01,random_state=1)
nb_Boost = AdaBoostClassifier(base_estimator=nb,n_estimators=100,learning_rate=0.01,random_state=1)
dt_Boost = AdaBoostClassifier(base_estimator=dt,n_estimators=100,learning_rate=0.01,random_state=1)
rf_Boost = AdaBoostClassifier(base_estimator=rf,n_estimators=100,learning_rate=0.01,random_state=1)
gb_Boost = GradientBoostingClassifier(n_estimators=100,learning_rate=0.01)
lgbm = LGBMClassifier(objective='binary',n_estimators=100,reg_alpha=2,reg_lambda=5,random_state=1,learning_rate=0.01,is_unbalance=True)

In [ ]:
kf = KFold(n_splits=3,shuffle=True,random_state=1)
model = LR_Boost
name = 'Boosted-LR'
#for model,name in zip([LR_Boost,nb_Boost,dt_Boost,rf_Boost,lgbm],['Boosted-LR','Boosted-NB','Boosted-DT','Boosted - Random Forest','LGBM']):
roc_acc = []
for train,test in kf.split(x,y):
    x_train = x[train,:]
    x_test = x[test,:]
    y_train = y[train]
    y_test = y[test]
    sm = SMOTE(random_state=1990, ratio=1.0)
    x_train, y_train = sm.fit_sample(x_train, y_train)
    model.fit(x_train,y_train)
    y_pred = model.predict(x_test)
    score = roc_auc_score(y_test,y_pred)
    fpr,tpr,_ = roc_curve(y_test,y_pred)
    roc_acc.append(auc(fpr,tpr))
Model.append(name)
ROC_AUC_Accuracy.append(np.mean(roc_acc))
print('The AUC Score for')
print('%s is %0.02f with variacne of (+/-) %0.5f'%(name,np.mean(roc_acc),np.var(roc_acc,ddof=1)))

In [ ]:
kf = KFold(n_splits=3,shuffle=True,random_state=1)
model = nb_Boost
name = 'Boosted-NB'
#for model,name in zip([LR_Boost,nb_Boost,dt_Boost,rf_Boost,lgbm],['Boosted-LR','Boosted-NB','Boosted-DT','Boosted - Random Forest','LGBM']):
roc_acc = []
for train,test in kf.split(x,y):
    x_train = x[train,:]
    x_test = x[test,:]
    y_train = y[train]
    y_test = y[test]
    sm = SMOTE(random_state=1990, ratio=1.0)
    x_train, y_train = sm.fit_sample(x_train, y_train)
    model.fit(x_train,y_train)
    y_pred = model.predict(x_test)
    score = roc_auc_score(y_test,y_pred)
    fpr,tpr,_ = roc_curve(y_test,y_pred)
    roc_acc.append(auc(fpr,tpr))
Model.append(name)
ROC_AUC_Accuracy.append(np.mean(roc_acc))
print('The AUC Score for')
print('%s is %0.02f with variacne of (+/-) %0.5f'%(name,np.mean(roc_acc),np.var(roc_acc,ddof=1)))

In [ ]:
kf = KFold(n_splits=3,shuffle=True,random_state=1)
model = dt_Boost
name = 'Boosted-DT'
#for model,name in zip([LR_Boost,nb_Boost,dt_Boost,rf_Boost,lgbm],['Boosted-LR','Boosted-NB','Boosted-DT','Boosted - Random Forest','LGBM']):
roc_acc = []
for train,test in kf.split(x,y):
    x_train = x[train,:]
    x_test = x[test,:]
    y_train = y[train]
    y_test = y[test]
    sm = SMOTE(random_state=1990, ratio=1.0)
    x_train, y_train = sm.fit_sample(x_train, y_train)
    model.fit(x_train,y_train)
    y_pred = model.predict(x_test)
    score = roc_auc_score(y_test,y_pred)
    fpr,tpr,_ = roc_curve(y_test,y_pred)
    roc_acc.append(auc(fpr,tpr))
Model.append(name)
ROC_AUC_Accuracy.append(np.mean(roc_acc))
print('The AUC Score for')
print('%s is %0.02f with variacne of (+/-) %0.5f'%(name,np.mean(roc_acc),np.var(roc_acc,ddof=1)))

In [ ]:
kf = KFold(n_splits=3,shuffle=True,random_state=1)
model = rf_Boost
name = 'Boosted-Random Forest'
#for model,name in zip([LR_Boost,nb_Boost,dt_Boost,rf_Boost,lgbm],['Boosted-LR','Boosted-NB','Boosted-DT','Boosted - Random Forest','LGBM']):
roc_acc = []
for train,test in kf.split(x,y):
    x_train = x[train,:]
    x_test = x[test,:]
    y_train = y[train]
    y_test = y[test]
    sm = SMOTE(random_state=1990, ratio=1.0)
    x_train, y_train = sm.fit_sample(x_train, y_train)
    model.fit(x_train,y_train)
    y_pred = model.predict(x_test)
    score = roc_auc_score(y_test,y_pred)
    fpr,tpr,_ = roc_curve(y_test,y_pred)
    roc_acc.append(auc(fpr,tpr))
Model.append(name)
ROC_AUC_Accuracy.append(np.mean(roc_acc))
print('The AUC Score for')
print('%s is %0.02f with variacne of (+/-) %0.5f'%(name,np.mean(roc_acc),np.var(roc_acc,ddof=1)))

In [ ]:
kf = KFold(n_splits=3,shuffle=True,random_state=1)
model = gb_Boost
name = 'Gradient Boost'
#for model,name in zip([LR_Boost,nb_Boost,dt_Boost,rf_Boost,lgbm],['Boosted-LR','Boosted-NB','Boosted-DT','Boosted - Random Forest','LGBM']):
roc_acc = []
for train,test in kf.split(x,y):
    x_train = x[train,:]
    x_test = x[test,:]
    y_train = y[train]
    y_test = y[test]
    sm = SMOTE(random_state=1990, ratio=1.0)
    x_train, y_train = sm.fit_sample(x_train, y_train)
    model.fit(x_train,y_train)
    y_pred = model.predict(x_test)
    score = roc_auc_score(y_test,y_pred)
    fpr,tpr,_ = roc_curve(y_test,y_pred)
    roc_acc.append(auc(fpr,tpr))
Model.append(name)
ROC_AUC_Accuracy.append(np.mean(roc_acc))
print('The AUC Score for')
print('%s is %0.02f with variacne of (+/-) %0.5f'%(name,np.mean(roc_acc),np.var(roc_acc,ddof=1)))

In [ ]:
kf = KFold(n_splits=3,shuffle=True,random_state=1)
model = lgbm
name = 'LGBM'
#for model,name in zip([LR_Boost,nb_Boost,dt_Boost,rf_Boost,lgbm],['Boosted-LR','Boosted-NB','Boosted-DT','Boosted - Random Forest','LGBM']):
roc_acc = []
for train,test in kf.split(x,y):
    x_train = x[train,:]
    x_test = x[test,:]
    y_train = y[train]
    y_test = y[test]
    sm = SMOTE(random_state=1990, ratio=1.0)
    x_train, y_train = sm.fit_sample(x_train, y_train)
    model.fit(x_train,y_train)
    y_pred = model.predict(x_test)
    score = roc_auc_score(y_test,y_pred)
    fpr,tpr,_ = roc_curve(y_test,y_pred)
    roc_acc.append(auc(fpr,tpr))
Model.append(name)
ROC_AUC_Accuracy.append(np.mean(roc_acc))
print('The AUC Score for')
print('%s is %0.02f with variacne of (+/-) %0.5f'%(name,np.mean(roc_acc),np.var(roc_acc,ddof=1)))

In [ ]:
final_result = pd.DataFrame({'Model - SMOTE Data':Model,'Accuracy':ROC_AUC_Accuracy})
final_result